# Quantitaive Analysis of Genetic Model

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

## Downasampling analysis

Generate barplots showing the diffrence between th target position and the median across different seeds of downsampling in terms of LOF scores.

In [47]:
def plot (ar, x_values, y_values, xerr_values=None,xlabel='', ylabel='',title ='', color ='#3288bd', path = ''):
    """
    plot function plot horizontal barplots from a dataframe columns
    """ 
    fig, axi= plt.subplots(figsize=(3,3), dpi = 300)
    ax = ar.plot(kind = "barh",x=x_values, y = y_values, legend = False, title = title, xerr = xerr_values , xlabel = ylabel,ax = axi)#     barp(arr) 
    ax.set_xlabel(xlabel)
    for s in ['top', 'right']:
        ax.spines[s].set_visible(False)
    plt.legend()
    plt.show()
#   plt.savefig(path+title ,  bbox_inches = "tight") 
#   plt.savefig(path+title ,  bbox_inches = "tight")              

In [34]:
path_inp ='/prj/Isabel_ONT_rRNA/Output/snakemake/18S/' # path to input as in the config file
output = '/prj/Isabel_ONT_rRNA/Output/snakemake/18S/analysis/MinION/downsampling_scores/lof_scores/' # path to outputs
thres = ['300','500','1000','5000', '10000'] # downsamplings coverage as set in config file
features = ['Mis', 'Mis+Del+Ins', 'Mis+Del+Ins_Context', 'MisContext+Del+Ins']
samples = { 'DIMT1L':1850,'DIMT1L1851':1851,'METTL5':1832, 'WBSCR22':1639} # list of analysis labels as set in the config file
seeds =['0','1','2','3','5','6','7','8','10','11','54321','42','25102021','20211025','123456789'] # list of seeds labels as set in the config file
baseline = 'median' # median or the top value from the first downsampling

In [ ]:
for feat in features:
    for sl in samples: 
        sample = sl.replace('DIMT1L1851','DIMT1L')
        ar =[]
        df =[]
        state = True
        for thre in thres: 
                tab = []
                for ij in seeds:
                    inp1 = '/prj/Isabel_ONT_rRNA/Output/snakemake/18S_'+ij+'/analysis/MinION/'+sample+ '.sampled'+thre+'/with_Parental_HCT116_Christiane_Zorbas_all_pass/'+ feat+'_lof_scores.csv'
                    t = pd.read_csv(inp1, sep = ',',header = 0)
                    maxi = t.sort_values('scores').reset_index(drop= True).loc[0,'scores']
                    if baseline == 'median':
                        median = t['scores'].median()
                        val = t.loc[(t.pos - samples[sl])==0,'scores'].values[0]
                        tab.append((median - val)/abs(maxi))
                    else:
                        if state == True: 
                            arr = t.sort_values('scores').reset_index(drop= True)
                            ref = arr[abs(arr.pos - samples[sl]) > 4].reset_index(drop = True).loc[0,'scores']
                            pos = arr[abs(arr.pos - samples[sl]) > 4].reset_index(drop = True).loc[0,'pos']
                            state = False
                        val = t.loc[(t.pos - samples[sl])==0,'scores'].values[0]
                        tab.append((ref - val)/abs(maxi))
                df.append([int(thre),np.mean(tab),np.std(tab)] )  
        ar= pd.DataFrame(df, columns=['cov', 'scores', 'std'])
        title = sample+"("+str(samples[sl])+")_median\n "+feat+"_LOF"
        plot(ar = ar, x_values= 'cov',y_values ="scores", xerr_values = 'std',xlabel = 'Coverage',  ylabel ='Distance', title = title)


Generate barplots showing the diffrence between th target position and the median in terms of combined features from JACUSA2 estimated scores.

In [ ]:
analysis= 'wt_ivt' # the pairwise analysis wt_ivt or ko_wt or ivt_wt
ref = 'NR_003286_RNA18SN5_'
for feat in features:
    for sl in samples: 
        sample = sl.replace('DIMT1L1851','DIMT1L')
        ar =[]
        df =[]
        for thre in thres: 
                tab = []
                for ij in seeds:
                    inp1 = '/prj/Isabel_ONT_rRNA/Output/snakemake/18S_'+ij+'/analysis/MinION/'+sample+ '.sampled'+thre+'/with_Parental_HCT116_Christiane_Zorbas_all_pass/Features_JACUSA2CALL2.csv'         
                    t = pd.read_csv(inp1, sep = ',',header = 0)
                    t =t.loc[t.label == analysis, :]
                    median = t[feat].median()
                    val = t.loc[(t.Ref_Pos == ref+ str(samples[sl])),feat].values[0]
                    maxi = t.sort_values(feat,ascending = False ).reset_index(drop= True).loc[0,feat]
                    mini = t.sort_values(feat,ascending = True ).reset_index(drop= True).loc[0,feat]
                    tab.append(val - median)
                
                df.append([int(thre),np.mean(tab),np.std(tab)] )  
        ar= pd.DataFrame(df, columns=['cov', feat, 'std'])
        title = sample+"("+str(samples[sl])+")_median\n "+feat+"_WTvsIVT"
        plot(ar = ar, x_values= 'cov',y_values = feat, xerr_values = 'std', xlabel = 'Coverage', ylabel = 'Distance', title = title)


## Analysis of Mixtures WT/KO

Generate barplots showing the fraction of detection of target position and its neighbors as outleirs across different seeds of mixing.

In [38]:
thres = ['0.25','0.5','0.75'] # list of mixing fraction of WT sample as set in the config file
features = ['MisContext+Del+Ins']
samples = { 'METTL5_Christiane_Zorbas_all_pass':1832, 'WBSCR22_Christiane_Zorbas_28S_18S_2zu1':1639} # KO sample name as set in the config file 
seeds =['1','2','3','5','6','7','8','10','11','54321','42','25102021','20211025','123456789'] # list of seeds as set in the config file

In [ ]:
for feat in features:
    for sl in samples: 
        sample = sl.split("_", 1)[0]
        ar =[]
        df =[]
        for thre in thres: 
                tab = pd.DataFrame([])
                for ij in seeds:
                    inp1 = path_inp+'analysis/MinION/'+sample+ '.sampled1000/DowS0/with_MixS'+ij+'/'+thre+'_Parental_HCT116_Christiane_Zorbas_all_pass'+sl+'/MisContext+Del+Ins_lof_scores.csv'         
                    t = pd.read_csv(inp1, sep = ',',header = 0)
                    t =t.loc[t.outlier == True, :]
                    zz = t.loc[(t.pos - samples[sl])== 0,:]
                    qq = t.loc[abs(t.pos - samples[sl]) < 3,:]
                    tab =tab.append([[qq.shape[0]/t.shape[0],zz.shape[0],qq.shape[0]]])
                df.append([float(thre),np.mean(tab[0]),np.mean(tab[1]),np.mean(tab[2])] )  
        ar= pd.DataFrame(df, columns=['thre','5mer outliers fraction', 'site '+str(samples[sl]), '5mer'])
        title =  sample+'_'+feat
        plot(ar, x_values='thre',y_values =['5mer outliers fraction', 'site '+str(samples[sl])], xlabel = 'Modification Rate', ylabel = "Fraction detected as outliers", title = title)

